In [82]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

In [102]:
G1=(lambda x1,x2,x3,x4:x1**2+x2**2+x3**2+x4**2)


In [103]:
G1(1,1,1,1)

4

In [104]:
def GetVectorF(G,r):
    
    
    v = np.zeros(1)
    
    v[0] = G(r[0],r[1],r[2],r[3])
        
    return v


In [105]:
GetVectorF(G1,[1,0,0,1])

array([2.])

In [106]:
def GetMetric(G,r):
    v = GetVectorF(G,r)
    return 0.5*np.linalg.norm(v)**2

In [107]:
def GetJacobian(G,r,h=1e-6):
    
    J = np.zeros((1,4))
    
    J[0,0] = (  G(r[0]+h,r[1],r[2],r[3]) - G(r[0]-h,r[1],r[2],r[3]) )/(2*h)
    J[0,1] = (  G(r[0],r[1]+h,r[2],r[3]) - G(r[0],r[1]-h,r[2],r[3]) )/(2*h)
    J[0,2] = (  G(r[0],r[1],r[2]+h,r[3]) - G(r[0],r[1],r[2]-h,r[3]) )/(2*h)
    J[0,3] = (  G(r[0],r[1],r[2],r[3]+h) - G(r[0],r[1],r[2],r[3]-h) )/(2*h)
        
    return J.T

In [108]:
GetMetric(G1,[1,-0.5,0,1])

2.53125

In [109]:
def GetFig(F,R,it):
    
    fig = plt.figure(figsize=(8,4))
    
    labels = ['X1','X2','X3','X4']
    
    ax = fig.add_subplot(1,2,1)
    ax1 = fig.add_subplot(1,2,2)

    ax.set_title('Metric: %.20f' %(F[it]))

    ax.plot(F[:it])
    ax.set_xlabel('%.0f' %(it))
    ax.set_yscale('log')
    ax1.plot(R[:it],label=labels)
    ax1.set_xlabel('%.0f' %(it))
    ax1.legend(loc=0)
    
    plt.show()

In [110]:
def GetSolve(G,r,lr=1e-3,epochs=int(1e5),error=1e-7):
    
    d = 1
    it = 0
    Vector_F = np.array([])
    
    R_vector = np.array(r)
    
    while d > error and it < epochs:
        
        CurrentF = GetMetric(G,r)
        
        J = GetJacobian(G,r)
        
        GVector = GetVectorF(G,r)
        
        #Machine Learning
        r -= lr*np.dot(J,GVector) 
        
        R_vector = np.vstack((R_vector,r))
        
        NewF = GetMetric(G,r)
        
        
        Vector_F = np.append(Vector_F,NewF)
        
        d = np.abs( CurrentF - NewF )/NewF
        
        
        if it%500 == 0:
            
            #print(it,d)
            clear_output(wait=True)
            GetFig(Vector_F,R_vector,it)
            time.sleep(0.01)
        it += 1
        
    if d < error:
        print(' Entrenamiento completo ', d, 'iteraciones', it)
        
    if it == epochs:
        print(' Entrenamiento no completado ')
        
    return r,it,Vector_F,R_vector

In [111]:
xsol,it,F,R = GetSolve(G1,[0.5,0.5,0.5,0.5],lr=1e-3)

KeyboardInterrupt: 

In [ ]:
xsol

In [ ]:
xsol,it,F,R = GetSolve(G1,[0,0,0,0],lr=1e-3)

In [ ]:
xsol